In [2]:
from download_data import get_titles_list

r = get_titles_list()

In [4]:
r

{'titles': [{'number': 1,
   'name': 'General Provisions',
   'latest_amended_on': '2022-12-29',
   'latest_issue_date': '2024-05-17',
   'up_to_date_as_of': '2025-04-11',
   'reserved': False},
  {'number': 2,
   'name': 'Federal Financial Assistance',
   'latest_amended_on': '2025-01-03',
   'latest_issue_date': '2025-01-06',
   'up_to_date_as_of': '2025-04-11',
   'reserved': False},
  {'number': 3,
   'name': 'The President',
   'latest_amended_on': '2015-03-17',
   'latest_issue_date': '2024-05-17',
   'up_to_date_as_of': '2025-04-11',
   'reserved': False},
  {'number': 4,
   'name': 'Accounts',
   'latest_amended_on': '2024-07-18',
   'latest_issue_date': '2024-07-18',
   'up_to_date_as_of': '2025-04-11',
   'reserved': False},
  {'number': 5,
   'name': 'Administrative Personnel',
   'latest_amended_on': '2025-03-24',
   'latest_issue_date': '2025-03-24',
   'up_to_date_as_of': '2025-04-11',
   'reserved': False},
  {'number': 6,
   'name': 'Domestic Security',
   'latest_amend